# Статическое квантование

Простой и наглядный пример для демонстрации основных концепций статического квантования.

Оригинал статьи с кодом можно найти [тут](https://ninjalabo.ai/blogs/pytorch_staticq.html).

Это пример лучше запускать либо на компьютере с GPU от NVidia, либо в [Google Colab](https://colab.research.google.com/).

In [ ]:
!pip3 install torch fastai

In [ ]:
import torch

print(f"Torch version {torch.__version__}")

# Let's make sure GPU is available!
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
from fastai.vision.all import *

import torch
from torch.ao.quantization import get_default_qconfig_mapping
import torch.ao.quantization.quantize_fx as quantize_fx
from torch.ao.quantization.quantize_fx import convert_fx, prepare_fx

Создадим класс __Quantizer__ для квантизации модели PyTorch. Для экспериментов мспользуем датасет Imagenette2-320 и модель ResNet18. И для удобства воспользуемся алгоритмами из  Fastai.

In [ ]:
class Quantizer():
    def __init__(self, backend="x86"):
        self.qconfig = get_default_qconfig_mapping(backend)
        torch.backends.quantized.engine = backend

    def quantize(self, model, calibration_dls):
        x, _ = calibration_dls.valid.one_batch()
        model_prepared = prepare_fx(model.eval(), self.qconfig, x)
        with torch.no_grad():
            _ = [model_prepared(xb.to('cpu')) for xb, _ in calibration_dls.valid]

        return model_prepared, convert_fx(model_prepared)

In [ ]:
path = untar_data(URLs.IMAGENETTE_320, data=Path.cwd()/'data')
dls = ImageDataLoaders.from_folder(path, valid='val', item_tfms=Resize(224),
                                   batch_tfms=Normalize.from_stats(*imagenet_stats))
learn = vision_learner(dls, resnet18)
model_prepared, qmodel = Quantizer("qnnpack").quantize(learn.model, learn.dls)

При статическом квантовании масштабный коэффициент и сдвиг нуля для весов и активаций определяются после калибровки модели, но до вывода результатов. Мы используем квантование по тензорам, а это значит, что есть единый масштабный коэффициент и сдвиг нуля применяются равномерно ко всем элементам каждого тензора слоя.

Модель __model_prepared__ запоминает диапазоны активаций по всему набору валидационных данных. Значит, мы можем посчитать масштабный коэффициент и сдвиг нуля.

Для сохранения диапазонов активаций используется __HistogramObserver__. Этот элемент появляется в модели благодаря применению функций __prepare_fx__ и __convert_fx__.

In [ ]:
# Example activation quantization parameters
for i in range(3):
    attr = getattr(model_prepared, f"activation_post_process_{i}")
    scale, zero_p = attr.calculate_qparams()
    print("{}\nScaling Factor: {}\nZero Point: {}\n".format(attr, scale.item(), zero_p.item()))

Обратите внимание, что слои Conv2d объединяются со слоями ReLU в квантованные слои QuantizedConvReLU2d.

В PyTorch, чтобы избежать избыточных процессов квантования и деквантования между слоями, пакетная нормализация встраивается в предыдущий слой (свертывание пакетной нормализации), а слой ReLU объединяется со следующим за ним слоем.

In [ ]:
qmodel

Взглянем внимательно на первый слой квантованной модели (Conv2d + ReLU).

In [ ]:
layer = qmodel._modules['0']._modules['0']
print(layer)
print("Weight Scale: {}, Weight Zero Point: {}".format(layer.weight().q_scale(),
                                                       layer.weight().q_zero_point()))
print("Output Scaling Factor: {}, Output Zero Point: {}\n".format(layer.scale, 
                                                                  layer.zero_point))

print("Example weights:", layer.weight()[0, 0, 0])
print("In integer representation:", layer.weight()[0, 0, 0].int_repr())

Теперь запустим инференс квантованной модели и сравним выходы первого сверточного слоя с фактическим результатом.

In [ ]:
layer_input = None
layer_output = None

def hook_fn(module, input, output):
    global layer_output, layer_input
    layer_input = input
    layer_output = output

img = torch.rand([1, 3, 224, 224])
hook = qmodel._modules['0']._modules['0'].register_forward_hook(hook_fn)
output = qmodel(img)
hook.remove()
print("Example input:", layer_input[0][0,0,0,:10].int_repr())
print("Example output:", layer_output[0,0,0,:10].int_repr())

In [ ]:
import numpy as np

def quantize(x, qparams, itype):
    xtype = torch.iinfo(itype)
    return torch.clamp(torch.round(x / qparams[0]) + qparams[1], min=xtype.min, max=xtype.max)

def dequantize(x, qparams):
    return (x - qparams[1]) * qparams[0]

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return torch.tensor(col)

# first use im2col, which is efficient way to perform Conv2d operation
inp = im2col(img, 7, 7, 2, 3).float()
# quantize input values using input scale and zero point
inp = quantize(inp, [layer_input[0].q_scale(), layer_input[0].q_zero_point()], torch.uint8)
# get quantized weights, weight scale and quantize biases
w = qmodel._modules['0']._modules['0'].weight().int_repr().reshape(64, -1).float()
sw = qmodel._modules['0']._modules['0'].weight().q_scale()
b = quantize(qmodel._modules['0']._modules['0'].bias(),
             [layer_input[0].q_scale() * sw, 0], torch.int32)
b = b.reshape(1,64,1,1).detach()
# calculate matmul in Conv2d and add biases
out = (w @ (inp.T - layer_input[0].q_zero_point())).view(1,64,112,112) + b
# dequantize, perform ReLU and quantize based on output scale and zero point
out = out * sw * layer_input[0].q_scale()
out = torch.relu(out)
out = quantize(out, [layer_output.q_scale(), layer_output.q_zero_point()], torch.uint8)

In [ ]:
torch.allclose(out, layer_output.int_repr().float())
print("Output: ", out[0, 0, 0, :10])